In [4]:
model = "llama3.2:1b"

#### Task 1: Simple Chain with Retrieval

**Objective:**

Implement a simple RAG chain with ChatOllama, HuggingFaceEmbeddings and Chroma. 

Process: 

1. Retrieve documents from chroma db based on query
2. Invoke chain with retrieved documents as input

**Task Description:**

- load llm model via ollama
- load embedding model via ollama with `ollama pull pull bge-m3` (if not yet done)
- create chroma db client
- create prompt template for summarization
- create simple chain with following steps: retrieved documents, prompt, model, output parser
- create query and perform similarity search with a query
- invoke chain and pass retrieved documents to the chain


**Useful links:**

- [RAG with Ollama](https://python.langchain.com/v0.2/docs/tutorials/local_rag/)
- [Streaming in Langchain](https://python.langchain.com/docs/concepts/streaming/)


In [5]:
from langchain_ollama import ChatOllama

# ADD HERE YOUR CODE
model = ChatOllama(model=model)

In [6]:
from langchain_ollama import OllamaEmbeddings

# ADD HERE YOUR CODE
embedding_model = OllamaEmbeddings(
    model="bge-m3",
    )

In [7]:
from langchain_chroma import Chroma
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

# Create a collection
# ADD HERE YOUR CODE
collection = client.get_or_create_collection("ai_model_book")  

# Create chromadb
# ADD HERE YOUR CODE
vector_db_from_client = Chroma(client=client, collection_name="ai_model_book", embedding_function=embedding_model)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given


In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | StrOutputParser()

In [9]:
search_query = "Types of Machine Learning Systems"

# ADD HERE YOUR CODE
# Perform vector search
docs = vector_db_from_client.similarity_search(search_query)

print(docs)

[Document(metadata={'page': 33, 'source': './AI_Book.pdf'}, page_content='Types of Machine Learning Systems\nThere are so many different types of Machine Learning systems that it is useful to\nclassify them in broad categories based on:\n Whether or not they are trained with human supervision (supervised, unsuper\nvised, semisupervised, and Reinforcement Learning)\n Whether or not they can learn incrementally on the fly (online versus batch\nlearning)\n Whether they work by simply comparing new data points to known data points,\nor instead detect patterns in the training data and build a predictive model, much\nlike scientists do (instance-based versus model-based learning)\nThese criteria are not exclusive; you can combine them in any way you like. For\nexample, a state-of-the-art spam filter may learn on the fly using a deep neural net\nwork model trained using examples of spam and ham; this makes it an online, model-\nbased, supervised learning system.\nLets look at each of these cr

In [10]:
chain.invoke(docs)

'Based on the retrieved documents, the main themes in these categorized types of machine learning systems are:\n\n**Classification based on Training Data and Supervision Methods**\n\n* The document highlights that machine learning systems can be classified into four major categories:\n\t+ Supervised learning (with labeled data)\n\t+ Unsupervised learning (without labeled data)\n\t+ Semisupervised learning (hybrid of supervised and unsupervised learning)\n\t+ Reinforcement learning (no direct supervision, but through trial-and-error)\n\n**Different Approaches to Learning from Data**\n\n* The document emphasizes that machine learning systems can learn incrementally on the fly or batch-wise\n* It also highlights that some systems can be classified as instance-based (detecting patterns in training data) or model-based (building predictive models like scientists do)\n* Another example mentioned is a state-of-the-art spam filter, which uses online, model-based supervised learning to classify

In [11]:
# Simple stream the chain output
for chunk in chain.stream(docs):
    print(chunk, end="", flush=True)

Based on the retrieved documents, the main themes that can be identified are:

1. **Classification of Machine Learning Systems**: The document discusses how to classify machine learning systems based on their characteristics, such as whether they are trained with human supervision, can learn incrementally, and work by comparing new data points or detecting patterns in training data.

2. **Machine Learning System Types**: The document identifies four main categories of machine learning systems:
   - Supervised Learning: where the algorithm receives labels to determine its accuracy.
   - Unsupervised Learning: where the algorithm analyzes the data without explicit labels.
   - Semisupervised Learning: where the algorithm has some labeled data but also other types of data (e.g., unlabeled or noisy).
   - Reinforcement Learning: where the algorithm learns by interacting with an environment and receiving rewards or penalties.

3. **Criteria for Classification**: The document emphasizes that

In [12]:
# More complex async event streaming
async for event in chain.astream_events(docs, version="v2"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush=True)

C:\Users\User\AppData\Local\Temp\ipykernel_5964\1927557568.py:2: LangChainBetaWarning: This API is in beta and may change in the future.
  async for event in chain.astream_events(docs, version="v2"):


The main themes in these retrieved documents about Machine Learning systems are:

1. **Classification of Machine Learning Systems**: The text discusses how to classify different types of Machine Learning systems based on their characteristics, such as whether they are trained with human supervision, can learn incrementally, and work by comparing new data points or detecting patterns.
2. **Categories for Classification**: The document outlines four major categories for classification: supervised learning (with labels), unsupervised learning (without labels), semisupervised learning (a combination of labeled and unlabeled data), and Reinforcement Learning (learning through trial and error).
3. **Characteristics of Machine Learning Systems**: The text highlights the importance of considering multiple characteristics when classifying Machine Learning systems, such as whether they are trained on the fly or in batches, and how they work compared to scientists.
4. **Importance of Supervision*

#### Task 2: Q&A with RAG

**Objective:**

Implement a Q/A retrieval chain with ChatOllama, HuggingFaceEmbeddings and Chroma

**Task Description:**

- create RAG-Q/A prompt template
- create retriever from vector db client (instead of manually passing in docs, we automatically retrieve them from our vector store based on the user question)
- create simple chain with following steps: retriever, formatting retrieved docs, user question, prompt, model, output parser
- create question for Q/A retrieval chain
- invoke chain and with question

**Useful links:**

- [RAG with Ollama](https://python.langchain.com/v0.2/docs/tutorials/local_rag/)

In [13]:
from langchain_core.runnables import RunnablePassthrough

prompt_template = """
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

<context>
{context}
</context>

Answer the following question:

{question}"""

# ADD HERE YOUR CODE
rag_prompt = ChatPromptTemplate.from_template(prompt_template)  

# ADD HERE YOUR CODE
retriever = vector_db_from_client.as_retriever()

# ADD HERE YOUR CODE
qa_rag_chain = ({
    "context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | model
    | StrOutputParser())

In [14]:
qa_rag_chain

{
  context: VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x00000225A6858410>)
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="\nYou are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\n\n<context>\n{context}\n</context>\n\nAnswer the following question:\n\n{question}"))])
| ChatOllama(model='llama3.2:1b', _client=<ollama._client.Client object at 0x0000022586064E90>, _async_client=<ollama._client.AsyncClient object at 0x0000022586064F50>)
| StrOutputParser()

In [15]:
question = "What is supervised learning?"

# ADD HERE YOUR CODE
qa_rag_chain.invoke(question)

'Supervised learning is a type of machine learning system where the training data includes desired solutions or labels that the algorithm aims to predict. This contrasts with unsupervised learning, where the training data is unlabeled and the algorithm learns without any external guidance.'

In [16]:
# More complex async event streaming
async for event in qa_rag_chain.astream_events(question, version="v2"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush=True)

Supervised learning is a type of machine learning where the training data includes desired solutions, called labels. This allows the algorithm to learn from the relationships between inputs and outputs, enabling it to make predictions or classifications on new, unseen data.

#### Alternative: Using pre-built ConversationalRetrievalChain Class

In [17]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

In [18]:
retriever = vector_db_from_client.as_retriever()
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [19]:
qa_chain = ConversationalRetrievalChain.from_llm(
    model, retriever=retriever, memory=memory, verbose=False
)

In [20]:
# More complex async event streaming
async for event in qa_chain.astream_events("What is supervised learning?", version="v2"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush=True)

Supervised learning is a type of machine learning where the algorithm is trained on labeled data, meaning that the data is already classified or tagged with the correct output. In other words, you provide the algorithm with examples of input and corresponding output labels, and then use those labeled examples to train the model.

In supervised learning, the goal is to learn a mapping between input features (e.g., numbers) and output labels (e.g., categories). The algorithm learns to predict or classify new, unseen data based on the patterns it has learned from the labeled data.

Here's an example of supervised learning in action:

Suppose you want to build a spam filter using machine learning. You collect a dataset of emails with their corresponding labels: "spam" if they are likely to be spam, and "not spam" otherwise. Your task is to train a model that can learn the patterns in this data and predict whether a new email will be spam or not.

In this case:

* The labeled data consists 

In [21]:
# More complex async event streaming
async for event in qa_chain.astream_events("Which algorithms can be used there?", version="v2"):
    kind = event["event"]
    if kind == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="", flush=True)

Here's the rephrased standalone question:

Which machine learning algorithms can be used for supervised learning?The following machine learning algorithms are typically used for supervised learning:

* k-Nearest Neighbors
* Linear Regression
* Logistic Regression
* Support Vector Machines (SVMs)
* Decision Trees and Random Forests
* Neural networks (although neural networks can also be unsupervised)